In [1]:
import argparse
import os
import os.path as osp
import shutil
import time
import warnings
warnings.filterwarnings('ignore')

import mmcv
import torch
from mmcv import Config
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction

from mmseg import digit_version
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import setup_multi_processes

In [2]:
# config 파일과 work_dir, epoch(model)을 설정해주세요.
cfg = Config.fromfile('../work_dirs/upernet_convnext_xlarge_fp16_512x512_160k_ade20k/upernet_convnext_xlarge_fp16_512x512_160k_ade20k.py')
cfg.work_dir = '../work_dirs/upernet_convnext_xlarge_fp16_512x512_160k_ade20k'
epoch = 'best_mIoU_epoch_60'


cfg.seed=2022
cfg.gpu_ids = [1]
cfg.model.train_cfg = None
cfg.data.test.test_mode = True

# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2023-01-04 22:23:36,692 - mmseg - INFO - Loaded 819 images


load checkpoint from local path: ../work_dirs/upernet_convnext_xlarge_fp16_512x512_160k_ade20k/best_mIoU_epoch_60.pth


In [3]:
output = single_gpu_test(model, data_loader) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 5.0 task/s, elapsed: 163s, ETA:     0s

In [5]:
import json
import pandas as pd

# 저장 위치를 설정하세요.
save_dir = "/opt/ml/input/code/submission"
csv_name = "convnext_xlarge_fold1_bestmIoU"

with open("/opt/ml/input/data/test.json", "r") as f:
    test = json.load(f)

# sample_submisson.csv 열기
submission = pd.read_csv(osp.join(save_dir,'sample_submission.csv'), index_col=None)

for id, out in zip(test['images'], output):
    # maxpooling
    input_size = 512
    output_size = 256
    bin_size = input_size // output_size

    out = out.reshape((1,512,512))
    result = out.reshape((1, output_size, bin_size, 
                            output_size, bin_size)).max(4).max(2)

    submission = submission.append({"image_id" : id['file_name'], "PredictionString" : " ".join(map(str, result.flatten().tolist()))}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(save_dir, index=False)

IsADirectoryError: [Errno 21] Is a directory: '/opt/ml/input/code/submission'